In [ ]:
%reload_ext autoreload
%autoreload 2

from drift.ftx import ftx_trade_to_target_position, recent_ftx_trades
from drift.drift import Drift, load_config, MARKET_INDEX_TO_PERP
import pandas as pd
import os
pd.options.plotting.backend = "plotly"

In [ ]:
USER_AUTHORITY = load_config() # TODO: replace with user wallet public key
USER_AUTHORITY

In [ ]:
drift = Drift(USER_AUTHORITY)
await drift.load()
history_df = await drift.load_history_df()
history_df.keys()

In [ ]:
drift.market_summary()

# trades

In [ ]:
trdf = history_df['trade'].copy().sort_index()
trdf['userAuthority'] = trdf['userAuthority'].astype(str)

duration = (trdf['fee'].index[-1] - trdf['fee'].index[0])
duration = duration.seconds/60/60

for x in ['fee','quoteAssetAmount']:
    trdf[x] /= 1e6
    trdf[x] = trdf[x].round(2)
for x in ['baseAssetAmount']:
    trdf[x] /= 1e13
for x in ['markPriceAfter','markPriceBefore','oraclePrice']:
    trdf[x] /= 1e10
    
# show volume of traders in 1024 most recent trades
toshow = trdf.groupby(['userAuthority', 'marketIndex'])\
[['baseAssetAmount','quoteAssetAmount','fee']].sum().sort_values('fee', ascending=False)\
.head(20)
print(int(duration*100)/100, 'hours')
# calculate interpolated daily fee spend
toshow['dailyFeeRate'] = (toshow['fee']/duration * 24).round(2)
toshow

# liquidations

In [ ]:
liqs = history_df['liquidation']
liqs = liqs[(liqs['marginRatio']!=0)]
liqs.columns
# (liqs['feeToLiquidator']/1e6).resample('D').sum().plot(kind='bar')
for col in ['totalCollateral', 'collateral','unrealizedPnl','feeToInsuranceFund','feeToLiquidator',
            'liquidationFee','baseAssetValue','baseAssetValueClosed']:
    liqs[col] /= 1e6
    
# liqs.sort_values('feeToLiquidator').to_csv("~/drift_liquidations_20211119.csv")

(liqs.loc['2021-11-18':])#['baseAssetValueClosed'])#.resample('1MIN').sum().plot()

((liqs.loc['2021-11-18'])['marginRatio']/1e3).hist()

(liqs['feeToInsuranceFund'].cumsum())#.plot()

# curve
adjustments to k and repeg events

In [ ]:
curvedffull = history_df['curve'].copy().sort_index()
for marketIndex in curvedffull.marketIndex.unique():
    curvedf = curvedffull[curvedffull.marketIndex == marketIndex]
    print(curvedf.columns)
    cdf = curvedf[['totalFee', 'totalFeeMinusDistributions','adjustmentCost']]/1e6
    fig = cdf.plot(title=MARKET_INDEX_TO_PERP[marketIndex])
    fig.show()

# deposits

In [ ]:
deposits = history_df['deposit'].loc['2021':]
d = deposits.direction[0]
assert('deposit' in str(d).lower())
deposit_ts = (deposits.apply(lambda x: x['amount']*-1 if x['direction']!=d else x['amount'],axis=1)/1e6)
deposit_ts.sort_index().cumsum().plot()

In [ ]:
tr = history_df['trade'].sort_index()
tr[tr.userAuthority.astype(str)!=USER_AUTHORITY].sort_index()
usrcheck = tr[tr.userAuthority.astype(str)==USER_AUTHORITY]
d = usrcheck.direction.values[0]
pos_ts = (usrcheck.apply(lambda x: x['baseAssetAmount']*-1 
                         if x['direction']!=d else x['baseAssetAmount'],axis=1)/1e13)
pos_ts.cumsum().plot()

# funding rate

In [ ]:
import numpy as np
frfull = history_df['fundingRate'].sort_index()

In [ ]:
for marketIndex in frfull.marketIndex.unique():
    fr = frfull[frfull['marketIndex']==marketIndex]  
    (fr[['oraclePriceTwap','markPriceTwap']]/1e10).replace(0,np.nan).dropna().plot()
    fr_hand = (fr[['oraclePriceTwap','markPriceTwap']].diff(axis=1)/1e10).iloc[:,-1]\
    .replace(0,np.nan)/24
    fr_prot = (fr[['cumulativeFundingRateLong', 'cumulativeFundingRateShort']]/1e14)\
    .replace(0,np.nan).diff()
    dfplt = pd.concat([fr_hand, fr_prot],axis=1)
    dfplt = (dfplt*100).mul(1/(fr['oraclePriceTwap']/1e10), axis=0).dropna().tail(7*24)
    
    dfplt = dfplt.rename({'cumulativeFundingRateLong':'long_funding_rate',
                  'cumulativeFundingRateShort':'short_funding_rate',
                  'markPriceTwap':'balanced_funding'
                 },axis=1)
    fig = dfplt.plot(title=MARKET_INDEX_TO_PERP[marketIndex]+' funding rate %')
    fig.show()

In [ ]:
# ?dfplt.plot

In [ ]:
mkt0 = drift.mkt_account.markets[0]
arbJ_position = drift.bot_position.positions[0].baseAssetAmount/1e13
total_position = (mkt0.baseAssetAmountLong - mkt0.baseAssetAmountShort)/1e13
arbJ_position/total_position

In [ ]:
arbJ_position = drift.bot_position.positions[0].baseAssetAmount/1e13
print(drift.base_asset_imbalance() - arbJ_position)
drift.base_asset_imbalance(), arbJ_position

In [ ]:
users_df = pd.DataFrame([x['account']['data'] for x in drift.all_users])
users_df['public_key'] = pd.Series([x['public_key'] for x in drift.all_users])

users_df['Address'] = users_df['authority'].astype(str)
for field in ['cumulativeDeposits', 'totalFeePaid', 'collateral', 'totalTokenDiscount']:
    users_df[field] = (users_df[field]/1e6).round(2)
user_summary_df = users_df[['Address', 'collateral', 'cumulativeDeposits', 'totalFeePaid']].sort_values('totalFeePaid',ascending=False)

In [ ]:
user_summary_df[user_summary_df.Address == ('4CLaLrTr7juGATt6FAaw8JksSNGdTEGaA78otYJbdHDs')]

In [ ]:
from drift.clearinghouse import ClearingHouseUser

In [ ]:
user = ClearingHouseUser(drift, USER_AUTHORITY)
await user.positions()

In [ ]:
cc = (users_df[['totalFeePaid', 'cumulativeDeposits']]).sort_values('cumulativeDeposits').reset_index(drop=True)#.iloc[:-1]

In [ ]:
cc.sum()

In [ ]:
(cc.iloc[:-1,]+1).pipe(np.log).plot(kind='bar')